## Introduction

The motivating setup for encryption systems is:

<img src="Crypto.png">

<img src="Crypto2.png">

We need two functions, Encrypt(m) -> c and Decrypt(c) -> m, that have two properties.

Correctness: for any possible message m, Decrypt(Encrypt(m)) = m.

Security: suppose encrypted message c = Encrypt(m). Then, it is "hard" to learn anything useful about m from c.

## Shared Key Cryptosystem

We also need the technique to be public without compromising the security property. The only secret should be the individual keys.

Thus, we add key k to the decrypt and encrypt functions as parameters.

This technique is easy to pull off if we can meet up in real life and exchange secret keys. However, this is not so simple when we have to exchange keys through an insecure channel.

### Diffie-Hellman Key Exchange

Suppose we have the situation in the picture above. Let Alice select a secret value a and let Bob select a secret value b. Agree on a "primitive root" g and a sufficiently large prime p. Then, exchange the values $m_{a} = g^{a}$  (mod p) and $m_{b} = g^{b}$  (mod p). Then, each person uses their secret value by exponentiating, that is, $k = (m_{a})^{b}$ (mod p) $ = (m_{b})^{a}$ (mod p). This generates the key $g^{ab}$ (mod p). 

A primitive root g of a prime p is a number where for any number m in {1,2,...,p-1}, there is some number n in {1,2,...,p-1} such that $g^{n} = m$ (mod p).

This means it is possible to generate any of the numbers in {1,2,...,p-1} with some secret value. 

The eavesdropping inteceptor cannot actually get the key, but both exchange parties manage to exchange the same key satisfying the correctness and security properties.

The only things exchanged through the network are {g, p, $m_{a}$, $m_{b}$}. Without adding a or b to that collection of information, there is no way (at the moment there are only exponential time algorithms) to derive the key besides brute force (which requires a large quantum computer) because $m_{a}$ effectively hides a and $m_{b}$ effectively hides b.

This cryptosystem relies on the "discrete logarithm problem" which is the problem that states: given {g, p, $m_{a}$}, find a.

This is an example of a function that is easy to compute, but the inverse is hard to compute.

### RSA Encryption

RSA encryption makes use of the idea of signatures.

Bob generates a public key and a private key. Then, gives out the public key for others to encrypt their message with. When Bob receives the encrypted message, he can decrypt it with his private key. Bob can "sign" his messages with his private key which lets people verify with the public key that Bob actually sent the message.

In RSA, the encrypt(verify) function is given by $E_{e}(M) = M^{e}$ (mod n) with the public key (e,n). The decrypt(sign) function is given by $D_{d}(C) = C^{d}$ (mod n) with the private key d. 

The correctness property is $D_{d}(E_{e}(M)) = M$. We can show this by $D_{d}(E_{e}(M)) = (E_{e}(M))^{d}$ (mod n) $= (M^{e}$ (mod n) $)^{d}$ (mod n) $ = M^{ed}$ (mod n) $ = M$ if e * d = 1. We need to choose e and d to have this property. 

$M^{ed}$ (mod n) $ = M \iff M^{ed-1}$ (mod n) $ = 1$

We need to develop some mathematical tools to generate these keys.

Fermat's Little Theorem: $a^{p-1} = 1$ (mod p) for a and p coprime.

Euler's Totient Function: $\varphi(n)$ is defined as the number of integers which are coprime to n between 1 and n.

For example: $\varphi(12) = 4$ since {1, 5, 7, 11} are all coprime to 12. Obvious corollary: for any prime p, $\varphi(p) = p-1$

Euler's Theorem: for a, n coprime, $a^{\varphi(n)} = 1$ (mod n).

$M^{k * \varphi(n) + 1}$ (mod n) $ = M \iff M^{k * \varphi(n)}$ (mod n) $ = 1$.
So the way we can choose the public key e and private key d is via $e * d = 1$ (mod $\varphi(n)$)

Assumption: n = p * q where p and q are two prime numbers. Then, $\varphi(n) = (p-1)(q-1)$. We will keep the generators p and q secret.

This gives use a easy formula for generating keys: $e * d = 1$ (mod $(p-1)(q-1)$) $\to$ $d = e^{-1}$ (mod $(p-1)(q-1)$)

$E_{e}(M) = M^{e}$ (mod n) is easy to compute using binary exponentiation algorithm.

Given {$E_{e}(M)$, e, n}, it is hard to compute M since we can't factor p and q easily from n in order to generate $(p-1)(q-1)$ to derive d from $d = e^{-1}$ (mod $(p-1)(q-1)$). All modern solutions are equivalent in time complexity to just brute force factoring.

However, it is easy(fast) to invert/decrypt given we know our private key d using the $D_{d}(C)$ function.

### Summary of one version of RSA key generation algorithm

Pick two really big prime numbers: p and q

Generate n = p * q

Pick public key exponent e

Compute private key d

Handout public key: (e,n)

Store private key in a safe location. Then, discard p and q and never let anyone know them.

## Example Python Implementation

Euclid's Algorithm for checking greatest common divisor. This allows us to check for coprime numbers by checking if gcd(a,b) == 1.

In [16]:
def gcd(a, b):
    if a < b:
        a,b = b,a
    if b == 0:
        return a
    else:
        return gcd(b, a%b)

Make an iterative implementation of this: (left as an exercise)

Coprime Check

In [ ]:
def is_coprime(a, b):
    if gcd(a,b) == 1:
        return True
    return False

Check if a number is prime

In [25]:
def is_prime(p):
    if p <= 1:
        return False
    elif p <= 3:
        return True
    elif p % 2 == 0 or p % 3 == 0:
        return False
    i = 5
    while i * i <= p:
        if p % i == 0 or p % (i+2) == 0:
            return False
        i += 6
    return True

Finding modular multiplicative inverse:   (g, x, y - (a // b) * x)

In [32]:
def inverse(a, n):
    for i in range(n):
        if (a*i) % n == 1:
            return i
    return 0

Generate keys

In [ ]:
def gen_keys(p, q):
    n = p * q
    totient = (p-1)*(q-1)
    e = generate_e(totient)
    d = inverse(e, totient)
    public = (e, n)
    private = (d, n)
    return public, private

def generate_e(t):
    from random import randint
    e = randint(1, t)
    while not is_coprime(e, t):              # only coprime numbers have multiplicative inverses
        e = randint(1, t)

Encrypt

In [31]:
def encrypt(public, message):      # keys are represented by pairs (exponent, modulus)
    e = public[0]
    n = public[1]
    c = pow(m, e, n)               # pow(base, exponent, modulus)
    return c

Decrypt

In [30]:
def decrypt(private, encrypted):
    d = private[0]
    n = private[1]
    decrypted = pow(encrypted, d, n)               # pow(base, exponent, modulus)
    return decrypted